In [2]:
sc.install_pypi_package("spark-nlp==2.4.5")
sc.install_pypi_package("nltk")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1607138080188_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/31/46/5c5a2bda407f693126386da5378f132e5e163fa6dfa46109548270348786/spark_nlp-2.4.5-py2.py3-none-any.whl


In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1607138080188_0004,pyspark,idle,Link,Link,
3,application_1607138080188_0006,pyspark,idle,Link,Link,
5,application_1607138080188_0008,pyspark,busy,Link,Link,
6,application_1607138080188_0009,pyspark,idle,Link,Link,✔


In [4]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version 
-------------------------- --------
beautifulsoup4             4.9.1   
boto                       2.49.0  
click                      7.1.2   
jmespath                   0.10.0  
joblib                     0.15.1  
lxml                       4.5.1   
mysqlclient                1.4.2   
nltk                       3.5     
nose                       1.3.4   
numpy                      1.16.5  
pip                        9.0.1   
py-dateutil                2.2     
python37-sagemaker-pyspark 1.3.0   
pytz                       2020.1  
PyYAML                     5.3.1   
regex                      2020.6.8
setuptools                 28.8.0  
six                        1.13.0  
soupsieve                  1.9.5   
spark-nlp                  2.4.5   
tqdm                       4.46.1  
wheel                      0.29.0  
windmill                   1.6

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ner")\
    .master("local[4]")\
    .config("spark.driver.memory","8G")\
    .config("spark.driver.maxResultSize", "2G") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.11:2.4.4")\
    .config("spark.kryoserializer.buffer.max", "500m")\
    .getOrCreate()

In [5]:
import sparknlp

spark = sparknlp.start()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
from pyspark.sql import functions as F

data_path = 's3://ctezna-doc-collection/preprocessed/docs/run-DataSink1-1-part-r-00000'
data = spark.read.csv(data_path, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
data.columns

In [ ]:
text_col = 'article'
article_text = data.select(text_col).filter(F.col(text_col).isNotNull())

In [ ]:
article_text.limit(5).show(truncate=90)

In [ ]:
from sparknlp.base import DocumentAssembler

documentAssembler = DocumentAssembler() \
     .setInputCol(text_col) \
     .setOutputCol('document')

In [ ]:
from sparknlp.annotator import Tokenizer

tokenizer = Tokenizer() \
     .setInputCols(['document']) \
     .setOutputCol('tokenized')

In [ ]:
from sparknlp.annotator import Normalizer

normalizer = Normalizer() \
     .setInputCols(['tokenized']) \
     .setOutputCol('normalized') \
     .setLowercase(True)

In [ ]:
from sparknlp.annotator import LemmatizerModel

lemmatizer = LemmatizerModel.pretrained() \
     .setInputCols(['normalized']) \
     .setOutputCol('lemmatized')

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

eng_stopwords = stopwords.words('english')

In [ ]:
from sparknlp.annotator import StopWordsCleaner

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemmatized']) \
     .setOutputCol('unigrams') \
     .setStopWords(eng_stopwords)

In [ ]:
from sparknlp.annotator import NGramGenerator

ngrammer = NGramGenerator() \
    .setInputCols(['lemmatized']) \
    .setOutputCol('ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

In [ ]:
from sparknlp.annotator import PerceptronModel

pos_tagger = PerceptronModel.pretrained('pos_anc') \
    .setInputCols(['document', 'lemmatized']) \
    .setOutputCol('pos')

In [ ]:
from sparknlp.base import Finisher

finisher = Finisher() \
     .setInputCols(['unigrams', 'ngrams', 'pos']) \

In [ ]:
from pyspark.ml import Pipeline

pipeline = Pipeline() \
     .setStages([documentAssembler,                  
                 tokenizer,
                 normalizer,                  
                 lemmatizer,                  
                 stopwords_cleaner, 
                 pos_tagger,
                 ngrammer,  
                 finisher])

In [ ]:
processed_review = pipeline.fit(article_text).transform(article_text)

In [ ]:
processed_review.limit(5).show()

In [ ]:
from pyspark.sql import types as T

udf_join_arr = F.udf(lambda x: ' '.join(x), T.StringType())
processed_review  = processed_review.withColumn('finished_pos', udf_join_arr(F.col('finished_pos')))

In [ ]:
pos_documentAssembler = DocumentAssembler() \
     .setInputCol('finished_pos') \
     .setOutputCol('pos_document')

In [ ]:
pos_tokenizer = Tokenizer() \
     .setInputCols(['pos_document']) \
     .setOutputCol('pos')

In [ ]:
pos_ngrammer = NGramGenerator() \
    .setInputCols(['pos']) \
    .setOutputCol('pos_ngrams') \
    .setN(3) \
    .setEnableCumulative(True) \
    .setDelimiter('_')

In [ ]:
pos_finisher = Finisher() \
     .setInputCols(['pos', 'pos_ngrams']) \

In [ ]:
pos_pipeline = Pipeline() \
     .setStages([pos_documentAssembler,                  
                 pos_tokenizer,
                 pos_ngrammer,  
                 pos_finisher])

In [ ]:
processed_review = pos_pipeline.fit(processed_review).transform(processed_review)

In [ ]:
processed_review.columns

In [ ]:
processed_review.select('finished_ngrams', 'finished_pos_ngrams').limit(5).show()

In [ ]:
def filter_pos(words, pos_tags):
    return [word for word, pos in zip(words, pos_tags) 
            if pos in ['JJ', 'NN', 'NNS', 'VB', 'VBP']]

udf_filter_pos = F.udf(filter_pos, T.ArrayType(T.StringType()))

In [ ]:
processed_review = processed_review.withColumn('filtered_unigrams',
                                               udf_filter_pos(F.col('finished_unigrams'), 
                                                              F.col('finished_pos')))

In [ ]:
processed_review.select('filtered_unigrams').limit(5).show(truncate=90)

In [ ]:
def filter_pos_combs(words, pos_tags):
    return [word for word, pos in zip(words, pos_tags) 
            if (len(pos.split('_')) == 2 and \
                pos.split('_')[0] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                 pos.split('_')[1] in ['JJ', 'NN', 'NNS']) \
            or (len(pos.split('_')) == 3 and \
                pos.split('_')[0] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                 pos.split('_')[1] in ['JJ', 'NN', 'NNS', 'VB', 'VBP'] and \
                  pos.split('_')[2] in ['NN', 'NNS'])]
    
udf_filter_pos_combs = F.udf(filter_pos_combs, T.ArrayType(T.StringType()))

In [ ]:
processed_review = processed_review.withColumn('filtered_ngrams',
                                               udf_filter_pos_combs(F.col('finished_ngrams'),
                                                                    F.col('finished_pos_ngrams')))

In [ ]:
processed_review.select('filtered_ngrams').limit(5).show(truncate=90)

In [ ]:
from pyspark.sql.functions import concat

processed_review = processed_review.withColumn('final', 
                                               concat(F.col('filtered_unigrams'), 
                                                      F.col('filtered_ngrams')))

In [ ]:
processed_review.select('final').limit(5).show(truncate=90)

In [ ]:
from pyspark.ml.feature import CountVectorizer

import time
start = time.time()

tfizer = CountVectorizer(inputCol='final', outputCol='tf_features')
tf_model = tfizer.fit(processed_review)
tf_result = tf_model.transform(processed_review)

end = time.time()
print('Train time: ', end - start)

In [ ]:
from pyspark.ml.feature import IDF

import time
start = time.time()

idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
idf_model = idfizer.fit(tf_result)
tfidf_result = idf_model.transform(tf_result)

end = time.time()
print('Train time: ', end - start)

In [ ]:
tfidf_result.show(1)

In [ ]:
from pyspark.ml.clustering import LDA

num_topics = 10
max_iter = 10

import time
start = time.time()

lda = LDA(k=num_topics, maxIter=max_iter, featuresCol='tf_idf_features')
lda_model = lda.fit(tfidf_result)

end = time.time()
print('Train time: ', end - start)

In [ ]:
vocab = tf_model.vocabulary

def get_words(token_list):
     return [vocab[token_id] for token_id in token_list]
       
udf_to_words = F.udf(get_words, T.ArrayType(T.StringType()))

In [ ]:
num_top_words = 20

topics = lda_model.describeTopics(num_top_words).withColumn('topicWords', udf_to_words(F.col('termIndices')))
topics.select('topic', 'topicWords').show(truncate=90)